In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import ConfigSpace as CS


import keras
from keras.datasets import cifar10
from os.path import abspath, join as path_join
PARENT_DIR =  path_join(os.getcwd(), "..")
sys.path.insert(0, PARENT_DIR)

from autoda.preprocessing import generate_batches
from autoda.data_augmentation import ImageAugmentation


import imgaug as ia
from imgaug import augmenters as iaa
from imgaug import parameters as iap

ia.seed(1)

labels_dict = {0:"airplane", 1: "automobile", 2: "bird", 3:"cat", 4:"deer", 5:"dog", 6:"frog",
               7:"horse", 8:"ship", 9:"truck"}
sample_config = ImageAugmentation.get_config_space().sample_configuration()
sample_config['augment_probability'] = 0.0

print(sample_config)

(x_train, y_train), _ = cifar10.load_data()
augmenter = ImageAugmentation(sample_config)
aug_batch, y = next(augmenter.apply_transform(x_train, y_train, batch_size=x_train.shape[0]))

print(x_train[0], "AUG", aug_batch[0])
#Visualizing CIFAR 10
n_rows, n_columns = 4, 5
fig, axes1 = plt.subplots(n_rows, n_columns,figsize=(10,10))
i = 0

from itertools import product

axes_iterator = iter(axes1)

indices = product(range(n_rows), range(n_columns))

for image, label in zip(aug_batch, np.reshape(y, (y.shape[0],))):
    label_name = labels_dict[label]
    row, column = next(indices)
    axis = axes1[row][column]
    axis.set_title("{}".format(label_name))
    axis.set_axis_off()
    axis.imshow(image)





In [ ]:

for image, label in zip(x_train[:20], np.reshape(y_train, (y_train.shape[0],))):
    label_name = labels_dict[label]
    row, column = next(indices)
    axis = axes1[row][column]
    axis.set_title("{}".format(label_name))
    axis.set_axis_off()
    axis.imshow(image)


In [ ]:
    from collections import namedtuple
    ParameterRange = namedtuple("ParameterRange", ["lower", "default", "upper"])

    def get_config_space(

            augment_probability=ParameterRange(lower=0, default=0.5, upper=1),
            pad_lower=ParameterRange(lower=0, default=0, upper=0.1),
            pad_upper=ParameterRange(lower=0.1, default=0.1, upper=0.3),
            pad_probability=ParameterRange(lower=0, default=0, upper=1),
            seed=None):

        config_space = CS.ConfigurationSpace(seed)

        hyperparameters = (

            CS.UniformFloatHyperparameter(
                "pad_lower",
                lower=pad_lower.lower,
                default=pad_lower.default,
                upper=pad_lower.upper,
            ),

            CS.UniformFloatHyperparameter(
                "pad_upper",
                lower=pad_upper.lower,
                default=pad_upper.default,
                upper=pad_upper.upper,
            ),

            CS.UniformFloatHyperparameter(
                "pad_probability",
                lower=pad_probability.lower,
                default=pad_probability.default,
                upper=pad_probability.upper,
            ),
            CS.UniformFloatHyperparameter(
                "augment_probability",
                lower=augment_probability.lower,
                default=augment_probability.default,
                upper=augment_probability.upper,
            ),
        )

        config_space.add_hyperparameters(hyperparameters)

        return config_space


In [ ]:
config = get_config_space().sample_configuration()
print(config)
seq = iaa.Sometimes(0.,
        iaa.Sequential([
            iaa.Sometimes(config['pad_probability'],
                          iaa.Pad(
                              percent=(
                                  config['pad_lower'],
                                  config['pad_upper']
                                  )
                              )
                          )
        ], random_order=True
        )
                        )


def apply_transform(x_train, y_train, batch_size=1):

    for batch_x, batch_y in zip(x_train, y_train):
        print(batch_x.shape, batch_y.shape)
                                
        # print("difference", np.sum(batch_x - seq.augment_images(batch_x)))
        yield seq.augment_image(batch_x), batch_y

(x_train, y_train), _ = cifar10.load_data()
aug_x, aug_y = next(apply_transform(x_train, y_train))

